# Computing distances for graph neural networks

## Part 1 - data preparation

Imports

In [4]:
import math
import os
import numpy as np
import pandas as pd
import requests
import torch
import copy
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim import Adamax
from tqdm import tqdm
from ray import tune
from ray.tune.schedulers.async_hyperband import ASHAScheduler
from datetime import datetime
from pathlib import Path
from random import sample
from typing import Tuple
from geopy.distance import geodesic
from glob import glob
from sklearn.preprocessing import normalize
from enum import Enum
from sklearn.linear_model import LinearRegression
from torch_geometric_temporal.nn.attention.stgcn import STConv
from torch_geometric_temporal.nn.recurrent import GCLSTM, DCRNN
from torch_geometric.nn import GCNConv
from torch.nn import ReLU, Linear, Module, BatchNorm1d, Dropout
from torch.utils.data.dataloader import DataLoader


Constants needed for data reading

In [5]:
DATA_FOLDER = "Data"
GRAPH_INFO_TXT = "d07_text_meta_2021_03_27.txt"
current_directory = os.getcwd()
path_raw_data = os.path.join(current_directory, DATA_FOLDER)
path_raw_data_graph_info_txt = os.path.join(path_raw_data, GRAPH_INFO_TXT)


Set total number of nodes (nb_days) from Metadata (may contain empty nodes).

In [6]:
def DataReader_get_number_of_nodes(path_raw_data_graph_info_txt):
    nodes_location = []
    skip = True
    with open(path_raw_data_graph_info_txt) as f:
        content = f.readlines()
        for line in content:
            if skip:
                skip = False
            else:
                line = line.split('\t')
                line = line[:-1]         # ID     #LAT     #LONG
                nodes_location.append([line[0], line[8], line[9]])

    return len(nodes_location)


In [7]:
total_num_nodes = DataReader_get_number_of_nodes(path_raw_data_graph_info_txt)
print(f'The total number of nodes are : {total_num_nodes}')

The total number of nodes are : 4904


Get nodes that have data as good_nodes and nodes that have empty data as empty_nodes from the data.

In [8]:
def DataReader_get_good_empty_nodes(path_raw_data,total_num_nodes):
    index = total_num_nodes
    empty_nodes = []
    good_nodes = []
    txtFiles = os.path.join(path_raw_data, "*", "*.txt")
    for file in glob(txtFiles):
        with open(file) as f:
            content = f.readlines()
            for line in content:
                line = line.split(',')
                line = [line1.replace("\n", "") for line1 in line]
                if not (line[9] == '' or line[10] == '' or line[11] == ''):
                    good_nodes.append((int)(line[1]))
                else:
                    empty_nodes.append((int)(line[1]))
                index -= 1
                if index == 0:
                    return (good_nodes,empty_nodes)

In [9]:
(good_nodes, empty_nodes) = DataReader_get_good_empty_nodes(
    path_raw_data, total_num_nodes)
print(f'Five examples of nodes that contain data: {good_nodes[:5]}')
print(f'Five examples of nodes that do not have data: {empty_nodes[:5]}')


Five examples of nodes that contain data: [715898, 715918, 715920, 715929, 715930]
Five examples of nodes that do not have data: [715900, 715901, 715903, 715904, 715905]


Set data and labels as X and Y variables from Data. They will contain good nodes and empty nodes as well

In [10]:
def DataReader_read_data(path_raw_data):
    X = []
    Y = []
    txtFiles = os.path.join(path_raw_data, "*", "*.txt")
    nb_days = 0
    for file in glob(txtFiles):
        with open(file) as f:
            print(f'Reading day {nb_days + 1}')
            content = f.readlines()
            for line in content:
                line = line.split(',')
                line = [line1.replace("\n", "") for line1 in line]
                if not (line[9] == '' or line[10] == '' or line[11] == ''):
                    Y.append((float)(line[11]))
                    X.append([(float)(line[9]), (float)(line[10])])
        nb_days += 1
        # TODO : code for debugging, delete when finished
        # ------------------------------------------------
        # if nb_days == 2:
        #     break
        # ------------------------------------------------
    X = normalize(np.array(X))
    Y = Y
    return X,Y,nb_days

In [11]:
X_data,Y_data,nb_days = DataReader_read_data(path_raw_data)

Reading day 1
Reading day 2
Reading day 3
Reading day 4
Reading day 5
Reading day 6
Reading day 7
Reading day 8
Reading day 9
Reading day 10
Reading day 11
Reading day 12
Reading day 13
Reading day 14
Reading day 15
Reading day 16
Reading day 17
Reading day 18
Reading day 19
Reading day 20
Reading day 21
Reading day 22


In [12]:
print(f'Total number of days read: {nb_days}')
print(f'Five examples of normalized input data: {X_data[:5]}')
print(f'Five examples of input data label: {Y_data[:5]}')

Total number of days read: 22
Five examples of normalized input data: [[9.99999984e-01 1.76732671e-04]
 [9.99999978e-01 2.08088231e-04]
 [9.99999974e-01 2.29906536e-04]
 [9.99999979e-01 2.03723400e-04]
 [9.99999976e-01 2.17045449e-04]]
Five examples of input data label: [70.4, 69.2, 66.0, 72.5, 69.8]


Get nodes geo location from Metadata. They will contain good nodes only

In [13]:
def DataReader_read_nodes_data(path_raw_data_graph_info_txt, good_nodes):

    nodes_location = []
    skip = True
    with open(path_raw_data_graph_info_txt) as f:
        content = f.readlines()
        for line in content:
            if skip:
                skip = False
            else:
                line = line.split('\t')
                line = line[:-1]
                if (int)(line[0]) in good_nodes:  # ID  #LAT    #LONG
                    nodes_location.append([line[0], line[8], line[9]])
    return nodes_location


In [14]:
nodes_location = DataReader_read_nodes_data(
    path_raw_data_graph_info_txt, good_nodes)
print(
    f'Five examples of nodes geolocation as well as their ID\'s {nodes_location[:5]}')

Five examples of nodes geolocation as well as their ID's [['715898', '33.880183', '-118.021787'], ['715918', '33.93311', '-118.091005'], ['715920', '33.938544', '-118.094941'], ['715929', '33.971707', '-118.123095'], ['715930', '33.971763', '-118.122905']]


Enumartion used for Dataset sizes

In [15]:
class Dataset(Enum):
    Experimental = 0
    ExperimentalManual = 1
    ExperimentalLR = 2
    Tiny = 3
    TinyManual = 4
    TinyLR = 5


In [16]:
class DatasetNodes(Enum):
    Experimental = [ 718292, 769496, 718291, 718290, 764567, 774279, 774278, 764671 ]
    Tiny = [
        775637, 718165, 776986, 759289, 774672, 760643, 774671, 717046, 718419,
        769105, 764026, 759280, 775636, 759385, 760635, 718166, 774685, 774658,
        716938, 776177, 763453, 718421, 717045, 768598, 717043, 716063, 717041,
        717040, 717039, 737184, 717042, 718335, 763458, 776981, 737158, 737313,
        769118, 772501, 718173, 764037, 763447, 763246, 718041, 763251, 763424,
        763429, 763434, 763439, 764032, 764418
    ]

In [17]:
class DatasetSize(Enum):
    Experimental = len(DatasetNodes.Experimental.value)
    Tiny = len(DatasetNodes.Tiny.value)

Function to set the nodes for the datasets such that they are the same throughout the project

More constants defined, as well as the nodes id's for the Experimental and Tiny dataset 

In [18]:
def Graph_get_nodes_for_dataset(dataset):
    experimental_all = [Dataset.Experimental,
                        Dataset.ExperimentalManual, Dataset.ExperimentalLR]
    tiny_all = [Dataset.Tiny, Dataset.TinyManual, Dataset.TinyLR]
    if dataset in experimental_all:
        return DatasetNodes.Experimental.value
    elif dataset in tiny_all:
        return DatasetNodes.Tiny.value


In [19]:
print(Graph_get_nodes_for_dataset(Dataset.Experimental))


[718292, 769496, 718291, 718290, 764567, 774279, 774278, 764671]


In [20]:
def Graph_get_number_of_nodes_for_dataset(dataset):
    experimental_all = [Dataset.Experimental,
                        Dataset.ExperimentalManual, Dataset.ExperimentalLR]
    tiny_all = [Dataset.Tiny, Dataset.TinyManual, Dataset.TinyLR]
    if dataset in experimental_all:
        return DatasetSize.Experimental.value
    elif dataset in tiny_all:
        return DatasetSize.Tiny.value


In [21]:
print(Graph_get_number_of_nodes_for_dataset(Dataset.Experimental))
print(Graph_get_number_of_nodes_for_dataset(Dataset.Tiny))

8
50


In [22]:
def Graph_extract_time(json):
    try:
        return float(json['routes']['distance'])
    except KeyError:
        return 0


In [23]:
def Graph_OSRM_loop(p1: tuple, p2: tuple) -> float:
    requestUrl = f'http://router.project-osrm.org/route/v1/driving/{p1[1]},{p1[0]};{p2[1]},{p2[0]}'

    try:
        response = requests.get(requestUrl)
    except:
        return -1
    if (response.status_code != 204
            and response.headers["content-type"].strip().startswith(
                "application/json")):
        try:
            responseJson = response.json()
        except:
            return 0
    else:
        return -1
    if responseJson['code'] == "Ok":
        routes = responseJson['routes']
        routes.sort(key=Graph_extract_time, reverse=True)
        shortest_distance = float(routes[0]['distance']) * (1 / 1000)
        return shortest_distance
    return


In [24]:
def Graph_OSRM(p1: tuple, p2: tuple) -> float:
    distance = Graph_OSRM_loop(p1, p2)
    while (distance == -1):
        distance = Graph_OSRM_loop(p1, p2)
    return distance


In [25]:
p1 = ('33.880183', '-118.021787')
p2 = ('33.93311', '-118.091005')
print(
    f'Geodesic distance between node id 715898 and node id 715918 is {geodesic(p1,p2)}')
print(
    f'Road distance between node id 715898 and node id 715918 is {Graph_OSRM(p1,p2)} km')


Geodesic distance between node id 715898 and node id 715918 is 8.68599891018443 km
Road distance between node id 715898 and node id 715918 is 8.8494 km


In [26]:
def Graph_compute_all_OSRM_and_Geodesic(nodes_location, path_distances):
    for dataset in [Dataset.Experimental, Dataset.Tiny]:

        name_OSRM = os.path.join(
            path_distances, f'distances_OSRM_{dataset.name}.npy')
        name_geodesic = os.path.join(
            path_distances, f'distances_Geodesic_{dataset.name}.npy')

        if (os.path.exists(name_OSRM) and os.path.exists(name_geodesic)):
            print(f'Distances already computed for {dataset.name}')
            continue

        nodes_ids = Graph_get_nodes_for_dataset(dataset)
        nodes_location_dataset = [node for node in nodes_location if (int)(node[0]) in nodes_ids]

        matrix_size = len(nodes_location_dataset)

        OSRM_array = np.zeros((matrix_size, matrix_size))
        geodesic_array = np.zeros((matrix_size, matrix_size))

        for i in range(matrix_size - 1):
            for j in range(i + 1, matrix_size):
                p1 = ((float)(nodes_location_dataset[i][1]),
                      (float)(nodes_location_dataset[i][2]))
                p2 = ((float)(nodes_location_dataset[j][1]),
                      (float)(nodes_location_dataset[j][2]))

                id_1 = (int)(nodes_location_dataset[i][0])
                id_2 = (int)(nodes_location_dataset[j][0])

                print(f'Computing distances for Id\'s {id_1} and {id_2}')

                OSRM_array[i][j] = Graph_OSRM(p1, p2)
                geodesic_array[i][j] = geodesic(p1, p2).km
                OSRM_array[j][i] = Graph_OSRM(p2, p1)
                geodesic_array[j][i] = geodesic(p2, p1).km

        np.save(name_OSRM, OSRM_array)
        np.save(name_geodesic, geodesic_array)

    return


In [27]:
path_processed_data = os.path.join(current_directory, "Processed")
if not os.path.exists(path_processed_data):
    os.makedirs(path_processed_data)

path_distances = os.path.join(path_processed_data, "Distances")
if not os.path.exists(path_distances):
    os.makedirs(path_distances)


In [28]:
print("Started computing distances...")
Graph_compute_all_OSRM_and_Geodesic(nodes_location, path_distances)
print("Finished computing distances...")


Started computing distances...
Distances already computed for Experimental
Distances already computed for Tiny
Finished computing distances...


In [29]:
array_distances_OSRM_Experimental = np.load(os.path.join(
    path_distances, f'distances_OSRM_Experimental.npy'))
array_distances_Geodesic_Experimental = np.load(os.path.join(
    path_distances, f'distances_Geodesic_Experimental.npy'))
print(array_distances_OSRM_Experimental)
print(array_distances_Geodesic_Experimental)


[[0.     2.3406 3.5436 7.6358 7.1201 3.2997 3.6629 5.1336]
 [1.8412 0.     1.203  5.2952 4.7795 0.9591 1.3223 3.7805]
 [4.7693 7.1099 0.     4.0921 3.5765 8.069  6.1248 5.4476]
 [0.6772 3.0178 4.2208 0.     7.7973 3.9769 4.3401 5.8108]
 [1.1929 3.5335 0.7643 0.5157 0.     4.4926 2.5484 3.1371]
 [5.0133 7.3539 0.2439 4.3361 3.8204 0.     6.3688 5.6916]
 [3.573  5.9136 3.4803 5.0002 4.4845 6.8727 0.     2.4172]
 [3.1712 5.5117 2.5385 6.6306 6.1149 6.4709 3.2839 0.    ]]
[[0.         0.02477349 1.20259522 0.67630258 1.19130412 0.95912002
  1.15459812 1.1201794 ]
 [0.02477349 0.         1.20137815 0.67577782 1.19058306 0.95783532
  1.15965723 1.12529727]
 [1.20259522 1.20137815 0.         0.52659971 0.02639492 0.24354287
  0.30303407 0.30835885]
 [0.67630258 0.67577782 0.52659971 0.         0.51500154 0.28354955
  0.52112531 0.48918423]
 [1.19130412 1.19058306 0.02639492 0.51500154 0.         0.23375099
  0.27704067 0.28200812]
 [0.95912002 0.95783532 0.24354287 0.28354955 0.23375099 0.
  

In [30]:
class DistanceType(Enum):
    Geodesic = 0
    OSRM = 1


In [31]:
def Graph_get_adjency_matrix_weight(ID1_index, ID2_index, epsilon, sigma, distanceType,distances_array) -> float:
    distance = distances_array[ID1_index][ID2_index]
    weight = math.exp(-((distance**2) / (sigma**2)))
    if weight >= epsilon:
        return weight
    else:
        return 0
     

In [32]:
edge_index_Experimental_manual = [[0, 1, 7, 4, 7, 5], [1, 2, 4, 3, 6, 4]]
edge_index_Tiny_manual = [[
    0, 0, 0, 1, 5, 5, 5, 9, 9, 9, 10, 10, 10, 6, 14, 15, 7, 13, 10, 11, 8,
    9, 4, 16, 5, 2, 20, 3, 22, 23, 24, 25, 26, 28, 29, 30, 31, 21, 32, 33,
    34, 36, 17, 38, 12, 39, 40, 41, 42, 44, 45, 46, 47, 48, 27, 35, 9
],
    [
    1, 2, 3, 4, 6, 7, 2, 4, 3, 12, 12, 11,
    4, 14, 15, 12, 13, 10, 11, 8, 2, 3, 16,
    17, 6, 20, 21, 22, 23, 24, 25, 26, 27,
    29, 30, 31, 7, 32, 33, 34, 35, 9, 37,
    5, 39, 40, 41, 42, 43, 45, 46, 47, 48,
    0, 19, 18, 36
]]


In [33]:
def Graph_get_info_for_Standard(nodes_location, epsilon, sigma, dataset, distanceType,array_distances):
    nodes = Graph_get_nodes_for_dataset(dataset)
    edge_index = []
    edge_weight = []
    nodes_location_manual = [node for node in nodes_location if (int)(node[0]) in nodes]
    num_nodes = len(nodes_location_manual)
    for i in range(num_nodes - 1):
        for j in range(i, num_nodes - 1):
            if i != j:
                weight = Graph_get_adjency_matrix_weight(i,j,epsilon, sigma, distanceType,array_distances)
                if weight > 0:
                    edge_index.append([i, j])
                    edge_weight.append(weight)
    edge_index = np.transpose(edge_index)
    return edge_index, edge_weight

In [34]:
def Graph_get_info_for_Manual(nodes_location, epsilon, sigma, dataset, distanceType,
            edge_index_Experimental_manual,edge_index_Tiny_manual,array_distances):
    
    edge_weight = []
    if dataset == Dataset.ExperimentalManual:
        edge_index = edge_index_Experimental_manual
    elif dataset == Dataset.TinyManual:
        edge_index = edge_index_Tiny_manual
    nodes = Graph_get_nodes_for_dataset(dataset)
    nodes_location_standard = [node for node in nodes_location if (int)(node[0]) in nodes]
    num_nodes = len(nodes_location_standard)
    nodes_info = np.zeros((num_nodes, 3))
    for i in range(num_nodes):
        np.where(nodes == (int)(nodes_location_standard[i][0]))[0]
        nodes_info[np.where(nodes == (int)(nodes_location_standard[i][0]))[0]] = nodes_location_standard[i]
    nodes_location_standard = np.array(nodes_info)
    for i in range(len(edge_index[0])):
        weight = Graph_get_adjency_matrix_weight(
            edge_index[0][i],edge_index[1][i], epsilon, sigma, distanceType,array_distances
        )
        edge_weight.append(weight)
    return edge_index, edge_weight


In [35]:
def Graph_get_top_3_nodes_for_node_with_LR(node, dataset, nodes_location, speed_vector):
    nodes_ids = Graph_get_nodes_for_dataset(dataset)
    ids_index = 0
    X_train = []
    Y_train = []
    nodes_used = []
    node_ids_order = [(int)(node[0]) for node in nodes_location]
    nodes_used_computed = False
    X_train_snapshot = []
    for speed in speed_vector:

        if node != node_ids_order[ids_index] and node_ids_order[ids_index] in nodes_ids:
            X_train_snapshot.append(speed)
            if not nodes_used_computed:
                nodes_used.append(node_ids_order[ids_index])

        if node == node_ids_order[ids_index]:
            Y_train.append(speed)

        ids_index += 1
        if ids_index == len(node_ids_order): 
            ids_index = 0
            X_train.append(X_train_snapshot)
            X_train_snapshot = []
            nodes_used_computed = True

    regression = LinearRegression(positive=True).fit(X_train, Y_train)
    coeffiecients = regression.coef_.tolist()
    results = zip(nodes_used, coeffiecients)
    sorted_results = sorted(results, key=lambda tup: tup[1], reverse=True)
    best = sorted_results[:3]
    return [result[0] for result in best]

In [36]:
print(Graph_get_top_3_nodes_for_node_with_LR(717042, Dataset.TinyLR, nodes_location, Y_data))

[759385, 718335, 764032]


In [37]:
def Graph_get_info_for_LR(nodes_location, epsilon, sigma, dataset, distanceType,array_distances, Y_data):
    edge_index = []
    edge_weight = []
    nodes_ids = Graph_get_nodes_for_dataset(dataset)
    for node in nodes_ids:
        nodes_relevant = Graph_get_top_3_nodes_for_node_with_LR(node, dataset, nodes_location, Y_data)
        for node_relevant in nodes_relevant:
            edge_index.append([nodes_ids.index(node_relevant),nodes_ids.index(node)])
            edge_weight.append(Graph_get_adjency_matrix_weight(
                    nodes_ids.index(node_relevant), nodes_ids.index(node),
                    epsilon, sigma, distanceType,array_distances))
    edge_index = [list(x) for x in set(tuple(x) for x in edge_index)]
    edge_index = np.transpose(edge_index)
    return edge_index, edge_weight

In [38]:
def Graph_save_graph(nodes_location, epsilon, sigma, dataset, distanceType, path_processed_data,
                    edge_index_Experimental_manual,edge_index_Tiny_manual,path_distances,Y_data):

        nodes = Graph_get_nodes_for_dataset(dataset)
        name_folder_weight = os.path.join(path_processed_data,'EdgeWeight')
        name_folder_index = os.path.join(path_processed_data,'EdgeIndex')
        if dataset in [Dataset.Experimental,Dataset.ExperimentalLR,Dataset.ExperimentalManual]:
            dataset_name = Dataset.Experimental.name
        if dataset in [Dataset.Tiny,Dataset.TinyLR,Dataset.TinyManual]:
            dataset_name = Dataset.Tiny.name
        array_distances = np.load(os.path.join(path_distances,f'distances_{distanceType.name}_{dataset_name}.npy'))

        if not os.path.exists(name_folder_weight):
            os.makedirs(name_folder_weight)

        if not os.path.exists(name_folder_index):
            os.makedirs(name_folder_index)

        postfix = f'{distanceType.name}_{epsilon}_{sigma}_{dataset.name}'

        name_weight = os.path.join(name_folder_weight,f'weight_{postfix}.npy')
        name_index = os.path.join(name_folder_index,f'index_{postfix}.npy')

        if os.path.exists(name_weight) and os.path.exists(name_index): 
            print(f'Graph already saved with configuration : epsilon = {epsilon}, sigma = {sigma}, size = {dataset.name}, distance = {distanceType.name}')
            return

        print(f'Saving graph with configuration : epsilon = {epsilon}, sigma = {sigma}, size = {dataset.name}, distance = {distanceType.name}')
        if (dataset == Dataset.ExperimentalManual or dataset == Dataset.TinyManual):
            edge_index, edge_weight = Graph_get_info_for_Manual(nodes_location, epsilon, sigma, dataset, distanceType,
                    edge_index_Experimental_manual,edge_index_Tiny_manual,array_distances)
        elif (dataset == Dataset.ExperimentalLR or dataset == Dataset.TinyLR):
            edge_index, edge_weight = Graph_get_info_for_LR(nodes_location, epsilon, sigma, dataset, distanceType,array_distances,Y_data)
        else:
            edge_index, edge_weight = Graph_get_info_for_Standard(nodes_location, epsilon, sigma, dataset, distanceType,array_distances)
        
        np.save(name_index, edge_index)
        np.save(name_weight, edge_weight)

In [39]:
EPSILON_ARRAY = [0.1, 0.3, 0.5, 0.7]
SIGMA_ARRAY = [1, 3, 5, 10]


In [40]:
for epsilon in EPSILON_ARRAY:
    for sigma in SIGMA_ARRAY:
        for distanceType in DistanceType:
            for dataset in Dataset:
                Graph_save_graph(nodes_location, epsilon, sigma, dataset, distanceType, path_processed_data,
                    edge_index_Experimental_manual,edge_index_Tiny_manual,path_distances,Y_data) 

Graph already saved with configuration : epsilon = 0.1, sigma = 1, size = Experimental, distance = Geodesic
Graph already saved with configuration : epsilon = 0.1, sigma = 1, size = ExperimentalManual, distance = Geodesic
Graph already saved with configuration : epsilon = 0.1, sigma = 1, size = ExperimentalLR, distance = Geodesic
Graph already saved with configuration : epsilon = 0.1, sigma = 1, size = Tiny, distance = Geodesic
Graph already saved with configuration : epsilon = 0.1, sigma = 1, size = TinyManual, distance = Geodesic
Graph already saved with configuration : epsilon = 0.1, sigma = 1, size = TinyLR, distance = Geodesic
Graph already saved with configuration : epsilon = 0.1, sigma = 1, size = Experimental, distance = OSRM
Graph already saved with configuration : epsilon = 0.1, sigma = 1, size = ExperimentalManual, distance = OSRM
Graph already saved with configuration : epsilon = 0.1, sigma = 1, size = ExperimentalLR, distance = OSRM
Graph already saved with configuration :

In [41]:
def arrange_data(data, num_nodes):
    r"""
        Function to arrange data. At a point it represents the temporal 
        state of the graph
    """
    New_Data = []
    for i in range((int)(len(data) / num_nodes)):
        Data = []
        for k in range(num_nodes):
            Data.append(data[(i * num_nodes) + k])
        New_Data.append(Data)
    return New_Data

In [42]:
def get_clean_data_by_nodes(dataset,nodes_location,X,Y):
    r"""
        Returns data for a specific datasize.
        Instance Function.
        Args:
            size : DatasetSize
        Returns a tuple of 2 lists with the Data and Labels
    """

    new_X = []
    new_Y = []
    nodes_index = 0
    nodes_ids = Graph_get_nodes_for_dataset(dataset)
    for _, tuple in enumerate(zip(X, Y)):
        if int(nodes_location[nodes_index][0]) in nodes_ids:
            new_X.append(tuple[0])
            new_Y.append([tuple[1]])
        nodes_index += 1
        if nodes_index == len(nodes_location):
            nodes_index = 0
    return new_X, new_Y

In [43]:
def save_data_for_train_LSTM(dataset,path_processed_data,nodes_location, X_data, Y_data):
    print(f"Saving data with configuration : size = {dataset.name}")

    X_all, Y_all = get_clean_data_by_nodes(dataset,nodes_location, X_data, Y_data)

    num_nodes = Graph_get_number_of_nodes_for_dataset(dataset)

    X_all = arrange_data(X_all, num_nodes)
    Y_all = arrange_data(Y_all, num_nodes)

    proccessed_data_path_model = os.path.join(path_processed_data,"LSTM")
    if not os.path.exists(proccessed_data_path_model):
        os.makedirs(proccessed_data_path_model)

    name_folder = os.path.join(proccessed_data_path_model,f'Data_{dataset.name}')
    if not os.path.exists(name_folder):
        os.makedirs(name_folder)

    for index, data in enumerate(X_all):
        name_x = os.path.join(name_folder, f'X_{index}.npy')
        if not os.path.exists(name_x):
            np.save(name_x, data)

    for index, data in enumerate(Y_all):
        name_y = os.path.join(name_folder, f'Y_{index}.npy')
        if not os.path.exists(name_y):
            np.save(name_y, data)

In [44]:
def save_proccess_data_STCONV(dataset,nb_days,path_processed_data,nodes_location, X_data, Y_data):
    batch_size = 8
    time_steps = 1
    interval_per_day = (int)(24 * 60 / 5)

    print(f"Saving data with configuration : size = {dataset.name}")

    Skip = (int)(time_steps / 2) * 2
    interval_per_day -= Skip

    X_all, Y_all = get_clean_data_by_nodes(dataset,nodes_location, X_data, Y_data)
    num_nodes = Graph_get_number_of_nodes_for_dataset(dataset)

    data_size_old = len(X_all)

    X_all = arrange_data(X_all, num_nodes)
    Y_all = arrange_data(Y_all, num_nodes)

    new_size = (int)(interval_per_day * nb_days / batch_size)
    data_size = len(X_all)
    if new_size * batch_size != data_size:
        difference = data_size - ((new_size - 1) * batch_size)
        X_all = X_all[:data_size - difference]
        Y_all = Y_all[:data_size - difference]
        new_size -= 1
    X_all = np.array(X_all).reshape(new_size, batch_size, time_steps, num_nodes,
                            2)
    Y_all = np.array(Y_all).reshape(new_size, batch_size, time_steps, num_nodes,
                            1)

    proccessed_data_path_model = os.path.join(path_processed_data,"STCONV")
    if not os.path.exists(proccessed_data_path_model):
        os.makedirs(proccessed_data_path_model)

    name_folder = os.path.join(proccessed_data_path_model,f'Data_{dataset.name}')
    if not os.path.exists(name_folder):
        os.makedirs(name_folder)

    for index, data in enumerate(X_all):
        name_x = os.path.join(name_folder, f'X_{index}.npy')
        if not os.path.exists(name_x):
            np.save(name_x, data)

    for index, data in enumerate(Y_all):
        name_y = os.path.join(name_folder, f'Y_{index}.npy')
        if not os.path.exists(name_y):
            np.save(name_y, data)

In [45]:
for dataset in [Dataset.Experimental,Dataset.Tiny]:
    save_proccess_data_STCONV(dataset,nb_days,path_processed_data,nodes_location,X_data,Y_data)
    save_data_for_train_LSTM(dataset,path_processed_data,nodes_location,X_data,Y_data)

Saving data with configuration : size = Experimental
Saving data with configuration : size = Experimental
Saving data with configuration : size = Tiny
Saving data with configuration : size = Tiny


Enumeration used for the folders name in which the processed data will be saved

In [46]:
class FoldersProcesedNames(Enum):
    STCONV = 0
    LSTM = 1
    EdgeWeight = 2
    EdgeIndex = 3
    Distances = 4

Function which will return true if the folders created after data processing exists

In [47]:
def Datareader_is_data_read(path_processed_data : str,folders_procesed_names : FoldersProcesedNames):
    for folder_name in folders_procesed_names:
        is_read_folder = os.path.isdir(os.path.join(path_processed_data, str(folder_name.name)))
        if not is_read_folder:
            return False
    return True

In [48]:
print(f'Data is read and stored: {Datareader_is_data_read(path_processed_data,FoldersProcesedNames)}')

Data is read and stored: True


In [49]:
class ModelType(Enum):
    r"""
        Enumeration for each model type.
            LSTM = 0
            STCONV = 1
            LinearRegression = 2
    """
    LSTM = 0
    DCRNN = 1
    STCONV = 2

In [50]:

class LossFunction():

    def Criterions():
        return [
            LossFunction.RMSE, LossFunction.MAPE, LossFunction.MAE,
            LossFunction.MSE
        ]

    def RMSE(y_pred, y_true):
        return torch.sqrt(torch.mean((y_pred - y_true)**2))

    def MAPE(y_pred, y_true):
        return torch.mean(torch.abs((y_true - y_pred) / y_true))

    def MAE(y_pred, y_true):
        return torch.mean(torch.abs((y_true - y_pred)))

    def MSE(y_pred, y_true):
        return torch.mean((y_true - y_pred)**2)

    #endregion

In [51]:
class STConvModel(Module):

    def __init__(self, node_features, num_nodes, hidden_channels, kernel_size,
                 ):
        super(STConvModel, self).__init__()

        self.STCONV = STConv(num_nodes=num_nodes,
                             in_channels=node_features,
                             hidden_channels=hidden_channels,
                             out_channels=node_features,
                             kernel_size=kernel_size,
                             K=1)
        self.linear = Linear(node_features, 1)
        self.ReLU = ReLU()

    def forward(self, x, edge_index, edge_weight):
        x = self.STCONV(x, edge_index, edge_weight)
        x = self.ReLU(x)
        x = self.STCONV(x, edge_index, edge_weight)
        x = self.ReLU(x)
        x = self.STCONV(x, edge_index, edge_weight)
        x = self.ReLU(x)
        x = self.linear(x)
        return x

In [52]:

class DCRNNModel(Module):

    def __init__(self, node_features, hidden_channels):
        super(DCRNNModel, self).__init__()

        self.DCRNN = DCRNN(in_channels=node_features,
                           out_channels=hidden_channels,
                           K=1)
        self.Conv1 = GCNConv(in_channels=hidden_channels,
                             out_channels=hidden_channels)
        self.BatchNorm1 = BatchNorm1d(num_features=hidden_channels)
        self.ReLU = ReLU()
        self.Dropout = Dropout()
        self.linear = Linear(hidden_channels, 1)

    def forward(self, x, edge_index, edge_weight):
        x = self.DCRNN(x, edge_index, edge_weight)
        x = self.Conv1(x, edge_index)
        x = self.ReLU(x)
        x = self.BatchNorm1(x)
        x = self.Dropout(x)
        x = self.linear(x)
        return x

In [53]:

class LSTMModel(Module):

    def __init__(self, node_features, hidden_channels):
        super(LSTMModel, self).__init__()

        self.GCLSTM1 = GCLSTM(in_channels=node_features,
                              out_channels=hidden_channels,
                              K=1)
        self.Conv1 = GCNConv(in_channels=hidden_channels,
                             out_channels=hidden_channels)
        self.BatchNorm1 = BatchNorm1d(num_features=hidden_channels)
        self.ReLU = ReLU()
        self.Dropout = Dropout()
        self.linear = Linear(hidden_channels, 1)

    def forward(self, x, edge_index, edge_weight):
        x = self.GCLSTM1(x, edge_index, edge_weight)
        x = self.Conv1(x[0], edge_index)
        x = self.ReLU(x)
        x = self.BatchNorm1(x)
        x = self.Dropout(x)
        x = self.linear(x)
        return x

In [54]:
class DatasetHelper():
    def __init__(self,
                 sigma: int,
                 epsilon: float,
                 size: DatasetSize,
                 distanceType: DistanceType,
                 proccessed_data_path,
                 folder_name,
                 device: str = 'cpu',
                 time_start: int = 0,
                 time_stop: float = -1):
                 
        self.proccessed_data_path = proccessed_data_path
        self.sigma = sigma
        self.epsilon = epsilon
        self.time_start = time_start
        self.time_stop = time_stop
        self.device = device
        self.dataset = dataset
        self.folder_name = folder_name
        self.distanceType = distanceType
        self.proccessed_data_path_model = os.path.join(self.proccessed_data_path, self.folder_name.name)
        self.__set_graph()
        self.__check_temporal_consistency()
        self.__set_snapshot_count()

    def __set_graph(self):
        name_weight = os.path.join(
            self.proccessed_data_path, 'EdgeWeight',
            f'weight_{self.distanceType.name}_{self.epsilon}_{self.sigma}_{self.dataset.name}.npy')
        self.edge_weight = np.load(name_weight, allow_pickle=True)

        name_index = os.path.join(
            self.proccessed_data_path, 'EdgeIndex',
            f'index_{self.distanceType.name}_{self.epsilon}_{self.sigma}_{self.dataset.name}.npy')
        self.edge_index = np.load(name_index, allow_pickle=True)

    def get_edge_index(self):
        if self.edge_index is None:
            return self.edge_index
        else:
            return torch.LongTensor(self.edge_index).to(self.device)

    def get_edge_weight(self):
        if self.edge_weight is None:
            return self.edge_weight
        else:
            return torch.FloatTensor(self.edge_weight).to(self.device)

    def __get_features(self, time_index: int):
        if (self.dataset in [Dataset.TinyManual, Dataset.TinyLR]):
            dataset = DatasetSize.Tiny
        elif (self.dataset in [Dataset.ExperimentalManual, Dataset.ExperimentalLR]):
            dataset = DatasetSize.Tiny
        else:
            dataset = self.dataset
        name_x = os.path.join(self.proccessed_data_path_model,
                              f"Data_{dataset.name}", f'X_{time_index}.npy')
        X = np.load(name_x)
        if X is None:
            return X
        else:
            return torch.FloatTensor(X).to(self.device)

    def __get_target(self, time_index: int):
        if (self.dataset in [Dataset.TinyManual, Dataset.TinyLR]):
            dataset = DatasetSize.Tiny
        elif (self.dataset in [Dataset.ExperimentalManual, Dataset.ExperimentalLR]):
            dataset = DatasetSize.Tiny
        else:
            dataset = self.dataset
        name_y = os.path.join(self.proccessed_data_path_model,
                              f"Data_{dataset.name}", f'Y_{time_index}.npy')
        Y = np.load(name_y)
        if Y is None:
            return Y
        else:
            if Y.dtype.kind == 'i':
                return torch.LongTensor(Y).to(self.device)
            elif Y.dtype.kind == 'f':
                return torch.FloatTensor(Y).to(self.device)

    def __getitem__(self, time_index: int):
        x = self.__get_features(time_index)
        y = self.__get_target(time_index)
        return x, y

    def __next__(self):
        if self.t < self.time_stop:
            snapshot = self.__getitem__(self.t)
            self.t = self.t + 1
            return snapshot
        else:
            self.t = self.time_start
            raise StopIteration

    def __iter__(self):
        self.t = self.time_start
        return self

    def __len__(self):
        return self.snapshot_count

    def __set_snapshot_count(self):
        if (self.dataset in [Dataset.TinyManual, Dataset.TinyLR]):
            dataset = DatasetSize.Tiny
        elif (self.dataset in [Dataset.ExperimentalManual, Dataset.ExperimentalLR]):
            dataset = DatasetSize.Tiny
        else:
            dataset = self.dataset
        self.snapshot_count = len(
            glob( os.path.join(self.proccessed_data_path_model, f"Data_{dataset.name}", "X_*.npy")))
    
    def __check_temporal_consistency(self):
        assert len(
            glob( os.path.join(self.proccessed_data_path_model, f"Data_{dataset.name}", "X_*.npy"))) == len(
            glob( os.path.join(self.proccessed_data_path_model, f"Data_{dataset.name}", "Y_*.npy"))), "Temporal dimension inconsistency."
    
    def get_dataset(self):
        train_ratio = 0.6
        val_ratio= 0.2
        test_ratio= 0.2

        time_train = int(train_ratio * self.snapshot_count)
        time_test = time_train + int(test_ratio * self.snapshot_count)

        train_iterator = DatasetHelper(self.sigma, self.epsilon, self.dataset,
                                       self.distanceType, self.proccessed_data_path, self.folder_name,
                                       self.device, 0, time_train)

        test_iterator = DatasetHelper(self.sigma, self.epsilon, self.dataset,
                                      self.distanceType,  self.proccessed_data_path, self.folder_name,
                                      self.device, time_train + 1, time_test)

        val_iterator = DatasetHelper(self.sigma, self.epsilon, self.dataset,
                                     self.distanceType,  self.proccessed_data_path, self.folder_name,
                                     self.device, time_test + 1,
                                     self.snapshot_count)

        return train_iterator, test_iterator, val_iterator

In [55]:
def test(best_model, dfResults, epoch, test_dataset,model_type) -> None:
    best_model.eval()
    loss = 0
    dataloader = DataLoader(test_dataset,
                            batch_size=1,
                            shuffle=False,
                            num_workers=0)
    edge_index = test_dataset.get_edge_index()
    edge_weight = test_dataset.get_edge_weight()
    MAE_loss = 0
    for criterion in LossFunction.Criterions():
        loss = 0
        for index, (x, y) in enumerate(dataloader):
            X = x[0]
            Y = y[0]
            y_hat = best_model(X, edge_index, edge_weight)
            loss += criterion(y_hat, Y)
            if criterion == LossFunction.MAE:
                MAE_loss += criterion(y_hat, Y)

        loss = loss / (index + 1)
        loss = loss.item()

        results = {
            "Model": str(model_type.name),
            "Epsilon": str(epsilon),
            "Sigma": str(sigma),
            "Dataset": str(dataset.name),
            "Criterion": str(criterion.__name__),
            "Loss": str(loss),
            "Epoch": str(epoch),
            "TestOrVal": "Test",
            "Trial": tune.get_trial_id()
        }
        dfResults = dfResults.append(results, ignore_index=True)

        if criterion == LossFunction.MAE:
            MAE_loss = MAE_loss / (index + 1)
            MAE_loss = MAE_loss.item()

    print("Best trial test set loss: {}".format(MAE_loss))
    return dfResults


In [56]:
def validate(dfResults, epoch,validation_dataset,model,model_type,dataset):
    model.eval()
    loss = 0
    dataloader = DataLoader(validation_dataset,
                            batch_size=1,
                            shuffle=False,
                            num_workers=0)
    edge_index = validation_dataset.get_edge_index()
    edge_weight = validation_dataset.get_edge_weight()
    MAE_loss = 0
    for criterion in LossFunction.Criterions():
        loss = 0
        for index, (x, y) in enumerate(dataloader):
            X = x[0]
            Y = y[0]
            y_hat = model(X, edge_index, edge_weight)
            loss += criterion(y_hat, Y)
            if criterion == LossFunction.MAE:
                MAE_loss += criterion(y_hat, Y)

        loss = loss / (index + 1)
        loss = loss.item()

        results = {
            "Model": str(model_type.name),
            "Epsilon": str(epsilon),
            "Sigma": str(sigma),
            "Dataset": str(dataset.name),
            "Criterion": str(criterion.__name__),
            "Loss": str(loss),
            "Epoch": str(epoch),
            "TestOrVal": "Validation",
            "Trial": tune.get_trial_id()
        }
        dfResults = dfResults.append(results, ignore_index=True)

        if criterion == LossFunction.MAE:
            MAE_loss = MAE_loss / (index + 1)
            MAE_loss = MAE_loss.item()

    return MAE_loss, dfResults

In [57]:
def train_validate_and_test(experiment_name,model,model_type,optimizer,scheduler,
    EarlyStoppingPatience,train_dataset,validation_dataset,test_dataset,nb_epoch,results_path):
    dfResults = pd.DataFrame(columns=[
        "Model", "Epsilon", "Sigma", "Dataset", "Criterion", "Loss", "Epoch", "Trial", "TestOrVal"
    ])
    train_model = model
    train_model.train()
    best_val_loss = np.inf
    val_model = model
    epoch_no_improvement = EarlyStoppingPatience
    dataloader = DataLoader(train_dataset,
                            batch_size=1,
                            shuffle=False,
                            num_workers=0)
    edge_index = train_dataset.get_edge_index()
    edge_weight = train_dataset.get_edge_weight()
    for epoch in tqdm(range(nb_epoch)):
        train_loss = 0
        for index, (x, y) in enumerate(dataloader):
            X = x[0]
            Y = y[0]
            y_hat = train_model(X, edge_index, edge_weight)
            loss = LossFunction.MAE(y_hat, Y)
            train_loss += loss
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        # Validation Step at epoch end
        val_loss, dfResults = validate(dfResults, epoch,validation_dataset,model,model_type,dataset)
        if val_loss < best_val_loss:
            val_model = copy.deepcopy(model)
            best_val_loss = val_loss
            epoch_no_improvement = EarlyStoppingPatience
        else:
            epoch_no_improvement -= 1

        if epoch_no_improvement == 0:
            print("Early stopping at epoch: {0}".format(epoch))
            dfResults = test(val_model, dfResults, epoch, test_dataset,model_type)
            if not os.path.exists(
                    os.path.join(results_path, experiment_name)):
                os.makedirs(
                    os.path.join(results_path, experiment_name))
            file_save = os.path.join( results_path, experiment_name,
                f"{model_type.name}_{dataset.name}_{distanceType.name}_{tune.get_trial_id()}.csv")
            dfResults.to_csv(path_or_buf=file_save, index=False)
            break

        train_loss = train_loss / (index + 1)
        scheduler.step(train_loss)
        print("Epoch {0} : Validation loss {1} ; Train loss {2};".format(
            epoch, val_loss, train_loss))

        # test step if this is the last epoch
        # Save dataframe results
        if epoch == nb_epoch - 1:
            dfResults = test(val_model, dfResults, epoch, test_dataset,model_type)
            if not os.path.exists(
                    os.path.join(results_path, experiment_name)):
                os.makedirs(
                    os.path.join(results_path, experiment_name))
            file_save = os.path.join(
                results_path, experiment_name,
                f"{model_type.name}_{dataset.name}_{distanceType.name}_{tune.get_trial_id()}.csv")
            dfResults.to_csv(path_or_buf=file_save, index=False)

        # Log to tune
        with tune.checkpoint_dir(step=epoch) as checkpoint_dir:
            path = os.path.join(checkpoint_dir, "checkpoint")
            torch.save(
                (val_model.state_dict(), optimizer.state_dict()),
                path)

        tune.report(loss=val_loss)
    return

In [58]:
def start_train(config, param, checkpoint_dir=None) -> None:
    folder_name = config["modelType"]
    if config["modelType"] == ModelType.DCRNN:
        folder_name = ModelType.LSTM.name

    train_dataset, validation_dataset, test_dataset = DatasetHelper(
                config["sigma"],
                config["epsilon"],
                config["dataset"],
                param["distanceType"],
                param["proccessed_data_path"],
                folder_name).get_dataset()
    if config["modelType"] == ModelType.STCONV:
        model = STConvModel(node_features=2,
                                     num_nodes=Graph_get_number_of_nodes_for_dataset(param["dataset"]),
                                     hidden_channels=32,
                                     kernel_size=1)
    elif config["modelType"] == ModelType.LSTM:
        model = LSTMModel(node_features=2, hidden_channels=32)
    elif config["modelType"] == ModelType.DCRNN:
        model = DCRNNModel(node_features=2, hidden_channels=32)
    optimizer = Adamax(model.parameters(), lr=param["learning_rate"])
    scheduler = ReduceLROnPlateau(optimizer,
                                           mode='min',
                                           factor=0.1,
                                           patience=5,
                                           threshold=0.00000001,
                                           threshold_mode='abs')
    train_validate_and_test(param["experiment_name"],model,config["modelType"],optimizer,scheduler,
    param["EarlyStoppingPatience"],train_dataset,validation_dataset,test_dataset,param["nb_epoch"],param["results_path"])

In [59]:
def trail_dirname_creator(trial):
        return f"{trial.config['modelType'].name}_{trial.config['dataset'].name}_{datetime.now().strftime('%d_%m_%Y-%H_%M_%S')}"

In [60]:
def hyperParameterTuning(dataset, model, distanceType, experiment_name,
    results_ray_path,epsilon_array,sigma_array,proccessed_data_path,results_path):

    nb_epoch = 30
    grace_period = 30
    reduction_factor = 3
    scheduler = ASHAScheduler(max_t=nb_epoch,
                                grace_period=grace_period,
                                reduction_factor=reduction_factor)
    learning_rate = 0.01
    num_features = 2
    EarlyStoppingPatience = 5
    nb_epoch
    param = {
        "learning_rate": learning_rate,
        "num_features": num_features,
        "EarlyStoppingPatience": EarlyStoppingPatience,
        "nb_epoch": nb_epoch,
        "experiment_name": experiment_name,
        "distanceType": distanceType,
        "proccessed_data_path": proccessed_data_path,
        "results_path": results_path
    }
    
    config = {
        "epsilon": tune.choice(epsilon_array),
        "sigma": tune.choice(sigma_array),
        "modelType": tune.choice([model]),
        "dataset" : tune.choice([dataset])
    }


    directory_experiment_ray = os.path.join(results_ray_path, experiment_name)

    num_samples = 16
    start_train.__name__ = f"{model.name}_{dataset.name}_{distanceType.name}"

    result = tune.run(tune.with_parameters(start_train, param=param),
                        local_dir=directory_experiment_ray,
                        trial_dirname_creator=trail_dirname_creator,
                        resources_per_trial={
                            "cpu": 8,
                            "gpu": 1
                        },
                        config=config,
                        metric="loss",
                        mode="min",
                        num_samples=num_samples,
                        scheduler=scheduler,
                        verbose=0)

    best_trial = result.get_best_trial("loss", "min", "last")

    print("Best trial config: {}".format(best_trial.config))
    print("Best trial for {} model final validation loss: {}".format(
        model.name, best_trial.last_result["loss"]))

In [61]:
results_path = os.path.join(current_directory,"Results")
if not os.path.exists(results_path):
    os.makedirs(results_path)

results_ray_path = os.path.join(current_directory,"Results-RAY")
if not os.path.exists(results_ray_path):
    os.makedirs(results_ray_path)

In [62]:
def start_learn(results_ray_path,path_processed_data,results_path):
    experiment_name = f'Experiment_{datetime.now().strftime("%d_%m_%Y-%H_%M_%S")}'
    directory_experiment_ray = os.path.join(results_ray_path, experiment_name)
    if not os.path.exists(directory_experiment_ray):
        os.makedirs(directory_experiment_ray)
        
    for distanceType in DistanceType:
        for dataset in DatasetSize:
            for model in ModelType:
                hyperParameterTuning(dataset, model, distanceType, experiment_name,
                    results_ray_path,EPSILON_ARRAY,SIGMA_ARRAY,path_processed_data,results_path)

In [63]:
start_learn(results_ray_path,path_processed_data,results_path)

2022-08-11 17:06:19,190	ERROR syncer.py:73 -- Log sync requires rsync to be installed.
2022-08-11 17:06:19,191	INFO trial.py:179 -- Creating a new dirname LSTM_Experimental_11_08_2022-17_06_18_2d90 because trial dirname 'LSTM_Experimental_11_08_2022-17_06_18' already exists.
  0%|          | 0/30 [00:00<?, ?it/s]


 pid=3664) Epoch 0 : Validation loss 12.77350902557373 ; Train loss 14.820738792419434;


  3%|▎         | 1/30 [01:56<56:27, 116.80s/it]


 pid=3664) Epoch 1 : Validation loss 14.073664665222168 ; Train loss 9.355070114135742;


  7%|▋         | 2/30 [03:52<54:10, 116.10s/it]


...

 pid=17248) Best trial test set loss: 12.670714378356934


(pid=17248) Windows fatal exception: access violation
(pid=17248) 


Best trial config: {'epsilon': 0.3, 'sigma': 1, 'modelType': <ModelType.LSTM: 0>, 'dataset': <DatasetSize.Experimental: 8>}
Best trial for LSTM model final validation loss: 13.773152351379395


2022-08-11 20:36:10,569	INFO trial.py:179 -- Creating a new dirname DCRNN_Experimental_11_08_2022-20_36_05_d866 because trial dirname 'DCRNN_Experimental_11_08_2022-20_36_05' already exists.
(pid=14832) 2022-08-11 20:36:12,449	ERROR function_runner.py:266 -- Runner Thread raised error.
(pid=14832) Traceback (most recent call last):
(pid=14832)   File "c:\Users\Sebi\anaconda3\envs\Dissertation\lib\site-packages\ray\tune\function_runner.py", line 260, in run
(pid=14832)     self._entrypoint()
(pid=14832)   File "c:\Users\Sebi\anaconda3\envs\Dissertation\lib\site-packages\ray\tune\function_runner.py", line 328, in entrypoint
(pid=14832)     return self._trainable_func(self.config, self._status_reporter,
(pid=14832)   File "c:\Users\Sebi\anaconda3\envs\Dissertation\lib\site-packages\ray\util\tracing\tracing_helper.py", line 449, in _resume_span
(pid=14832)     return method(self, *_args, **_kwargs)
(pid=14832)   File "c:\Users\Sebi\anaconda3\envs\Dissertation\lib\site-packages\ray\tune\fun

TuneError: ('Trials did not complete', [DCRNN_Experimental_Geodesic_13f6a_00000, DCRNN_Experimental_Geodesic_13f6a_00001, DCRNN_Experimental_Geodesic_13f6a_00002, DCRNN_Experimental_Geodesic_13f6a_00003, DCRNN_Experimental_Geodesic_13f6a_00004, DCRNN_Experimental_Geodesic_13f6a_00005, DCRNN_Experimental_Geodesic_13f6a_00006, DCRNN_Experimental_Geodesic_13f6a_00007, DCRNN_Experimental_Geodesic_13f6a_00008, DCRNN_Experimental_Geodesic_13f6a_00009, DCRNN_Experimental_Geodesic_13f6a_00010, DCRNN_Experimental_Geodesic_13f6a_00011, DCRNN_Experimental_Geodesic_13f6a_00012, DCRNN_Experimental_Geodesic_13f6a_00013, DCRNN_Experimental_Geodesic_13f6a_00014, DCRNN_Experimental_Geodesic_13f6a_00015])